In [4]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
data = load_iris()
df_iris = pd.DataFrame(data.data, columns = ['sepal length', 'sepal width', 'petal length', 'petal width'])
df_iris['class'] = data.target
df_iris.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [6]:
iris_model = LogisticRegression(solver='liblinear', multi_class='ovr')
iris_model.fit(df_iris.values, df_iris['class'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [7]:
iris_pred = iris_model.predict(df_iris.values)
accuracy_score(iris_pred, df_iris['class'])

1.0

In [8]:
X = df_iris.drop('class', axis=1).values
Y = df_iris['class'].values
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

In [9]:
iris_model = LogisticRegression(solver='liblinear', multi_class='ovr')
iris_model.fit(x_train, y_train)
y_pred = iris_model.predict(x_test)
accuracy_score(y_pred, y_test)

1.0

In [10]:
from sklearn import svm

clf = svm.SVC(gamma='scale')
clf.fit(x_train, y_train)  
accuracy_score(clf.predict(x_test), y_test)

0.9777777777777777

In [11]:
from sklearn.neural_network import MLPClassifier

nn_clf = MLPClassifier(activation='relu', max_iter=500, hidden_layer_sizes=(20, 30))
nn_clf.fit(x_train, y_train)
nn_clf.predict(x_test)

/home/xma/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


array([2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 1, 1, 1, 2, 2, 2, 0,
       2, 0, 1, 2, 1, 0, 1, 2, 1, 1, 2, 0, 0, 1, 0, 1, 2, 2, 0, 1, 2, 2,
       0])

In [14]:
nn_clf.predict_proba(x_test)

array([[1.33123157e-08, 1.29785495e-03, 9.98702132e-01],
       [9.96769169e-01, 3.23083054e-03, 1.05165163e-12],
       [5.14861869e-10, 2.43557422e-04, 9.99756442e-01],
       [9.99685461e-01, 3.14539306e-04, 2.34507745e-15],
       [9.47625244e-06, 2.29204796e-01, 7.70785727e-01],
       [9.53195276e-09, 2.53579608e-03, 9.97464194e-01],
       [9.99474611e-01, 5.25388847e-04, 6.97605681e-15],
       [9.96754267e-01, 3.24573269e-03, 1.13823306e-12],
       [3.83969033e-07, 3.27193465e-02, 9.67280270e-01],
       [9.96051949e-01, 3.94805137e-03, 2.66950615e-12],
       [9.98836634e-01, 1.16336570e-03, 1.22001084e-13],
       [7.26662036e-08, 5.34386205e-03, 9.94656065e-01],
       [9.96560847e-01, 3.43915316e-03, 1.60747224e-12],
       [9.98648719e-01, 1.35128106e-03, 9.60697140e-14],
       [2.15122858e-07, 1.20774322e-02, 9.87922353e-01],
       [4.27689384e-04, 9.98771784e-01, 8.00526217e-04],
       [7.93487685e-04, 9.98822573e-01, 3.83939328e-04],
       [4.13869127e-05, 8.20735

In [ ]:
accuracy_score(nn_clf.predict(x_test), y_test)

In [ ]:
nn_clf.predict(x_test)

In [2]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from keras.layers import Dense, Input, concatenate, Dropout
from keras.models import Model
from keras.optimizers import rmsprop

dataset = load_wine()

ensemble_num = 10 # number of sub-networks
bootstrap_size = 0.8 # 80% size of original (training) dataset
training_size = 0.8 # 80% for training, 20% for test

num_hidden_neurons = 10 # number of neurons in hidden layer
dropout = 0.25 # percentage of weights dropped out before softmax output (this prevents overfitting)

epochs = 200 # number of epochs (complete training episodes over the training set) to run
batch = 10 # mini batch size for better convergence

# get the holdout training and test set
temp = []
scaler = MinMaxScaler()
one_hot = OneHotEncoder() # one hot encode the target classes
dataset['data'] = scaler.fit_transform(dataset['data'])
dataset['target'] = one_hot.fit_transform(np.reshape(dataset['target'], (-1,1)) ).toarray()
for i in range(len(dataset.data)):
    temp.append([dataset['data'][i], np.array(dataset['target'][i])])

# shuffle the row of data and targets
temp = np.array(temp)
np.random.shuffle(temp)
# holdout training and test stop index
stop = int(training_size*len(dataset.data))

train_X = np.array([x for x in temp[:stop,0]])
train_Y = np.array([x for x in temp[:stop,1]])
test_X = np.array([x for x in temp[stop:,0]])
test_Y = np.array([x for x in temp[stop:,1]])

# now build the ensemble neural network
# first, let's build the individual sub-networks, each
# as a Keras functional model.
sub_net_outputs = []
sub_net_inputs = []
for i in range(ensemble_num):
    # two hidden layers to keep it simple
    # specify input shape to the shape of the training set
    net_input = Input(shape = (train_X.shape[1],))
    sub_net_inputs.append(net_input)
    y = Dense(num_hidden_neurons)(net_input)
    y = Dense(num_hidden_neurons)(y)
    y = Dropout(dropout)(y)
    sub_net_outputs.append(y) # sub_nets contains the output tensors

# now concatenate the output tensors
y = concatenate(sub_net_outputs)

# final softmax output layer
y = Dense(train_Y[0].shape[0], activation='softmax')(y)

# now build the whole funtional model
model = Model(inputs=sub_net_inputs, outputs=y)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

print("Begin training...")

# train the model
# model.fit( [train_X] * ensemble_num, train_Y,validation_data=[ [test_X] * ensemble_num, test_Y],
#           epochs=epochs, batch_size=batch)

/home/xma/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Begin training...
